# AbstractGEN ver.4

이 포스트는 6개로 이루어진 "AbstractGEN"의 5번째 포스트입니다.

이번 포스트의 내용을 3가지로 요약하면 다음과 같습니다. 
- bidirectional GRU Encoder
- Decoder with Attention Layer
- Teacher forcing

자세한 설명은 코드와 함께 첨부하였습니다. 

앞으로의 포스트는 다음과 같습니다:

0. AbstractGEN: DataAnalysis
1. AbstractGEN: Char-level RNN
2. AbstractGEN: Word-level RNN
3. AbstractGEN: Seq2seq
4. **AbstractGEN: Seq2seq with attention+teaching force**
5. AbstractGEN: Applying gpt-2

In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch.nn.functional as F

import pandas as pd
import tqdm

import random
import math
import time

import re
import collections
import itertools

import unicodedata
import string

## Text reshape

Raw data의 각 논문은 두 번의 enter('\n\n') , 제목과 초록은 한 번의 enter('\n')를 기준으로 나누어져 있었다. 

이를 각 논문은 한 번의 enter, 제목과 초록은 tap('\t')로 구분되도록 바꾸었다. 

이러한 형식을 택한 이유는 뒤의 PariDataset 코드에서 쌍을 만들기 쉽게 하기 위함이다.

In [ ]:
all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

def unicodeToAscii(s):
  return ''.join(
      c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn'
  )

def normalizeString(s):
  
  s = unicodeToAscii(s.lower().strip())
  s = re.sub(r"([.!?])", r" \1", s)
  s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
  return s

In [ ]:
lines = open("/content/acl_titles_and_abstracts.txt", encoding='utf-8').read().strip().split('\n')
data = [normalizeString(line) for line in lines]

dt = pd.DataFrame(data)


data_dict = []
for i in range(0, len(data),3):
  data_i = []
  data_i.append(data[i])
  data_i.append(data[i+1])
  data_dict.append(data_i)

df = pd.DataFrame(data_dict[:10000])
dp = pd.DataFrame(data_dict[-2000:])

dt.to_csv('titles_and_abstracts.txt', header=False, index=False)
dp.to_csv('abstract_test_2000.txt', sep='\t', header=False, index=False)
df.to_csv('abstract_train_10000.txt', sep='\t', header=False, index=False)

In [ ]:
df.head()

,0,1
0,evaluation technology from speaker identificat...,we propose a multi step system for the analysi...
1,acquisition of semantic classes for adjectives...,in this paper we present a clustering experime...
2,features for detecting hedge cues,we present a sequential labeling approach to h...
3,semantic extraction with wide coverage lexical...,we report on results of combining graphical mo...
4,evaluating a crosslinguistic grammar resource ...,this paper evaluates the lingo grammar matrix ...


In [ ]:
SEED = 1024

random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
FILTERS = "([~,\"':;)(])"
SHIFTS = re.compile("([?!.])")
CHANGE_FILTER = re.compile(FILTERS)

unk = 0
sos = 1
eos = 2

Filter는 데이터에서 삭제할 목록들, Shifts는 단어와 해당 목록 사이에 띄어쓰기를 할 목록을 의미한다. ' '를 삽입함으로써 "do."와 "do"가 별개의 단어로 취급되는 것을 막고, Shifts의 목록을 개별 단어로 여기게 한다. 

 unk는 단어 사전(훈련 데이터에 등장하는 모든 단어의 목록)에 존재하지 않는 단어를 의미한다. unkown의 약자로, 앞으로 숫자 0으로 표기한다. 

 sos는 input 데이터의 시작을 알린다. start of the sentence의 약자이다. 

 eos는 데이터의 끝을 알린다.  end of the sentence의 약자이다. 

해당 3가지는 우리가 사용한 seq2seq모델에서 사용되는 것으로, 뒤에 자세히 기술하겠다.

## Preprocessing

In [ ]:
def normalize(pair):
  pair = pair.lower()
  pair = CHANGE_FILTER.sub("", pair)
  pair = re.sub('[0-9]+', "", pair)
  pair = SHIFTS.sub(r" \1", pair)
  return pair

def parse_line(pair):
  pair = normalize(pair.strip())
  src, trg = pair.split('\t')
  src_tokens = src.strip().split()
  trg_tokens = trg.strip().split()
  return src_tokens, trg_tokens

def build_vocab(words): 
  counts = collections.Counter(words)
  sort_words = sorted(counts.items(),
                      key=lambda x:x[1], reverse = True)
  word_list = ["<UNK>", "<SOS>", "<EOS>"]+ [x[0] for x in sort_words]
  word_dict = dict((w,i) for i,w in enumerate(word_list))
  return word_list, word_dict

def word2tensor(words, word_dict, max_len, sos = False, padding = 0):
  words = words+['<EOS>']
  if sos: words = ['<SOS>'] + words

  words = [word_dict.get(w,0) for w in words]
  seq_len = len(words)
  if seq_len < max_len+1:
    words += [padding]*(max_len+1 -seq_len)
  return torch.tensor(words, dtype=torch.int64), seq_len


- normalize() 함수는 텍스트 전체를 받아, 모두 소문자로 바꾸고, 위에서 설명한 FILTER, SHIFTS를 수행한다.  

- parse_line() 함수는 텍스트에서 읽어온 각 line을 입력으로 받는다. 

예를 들어, 우리의 데이터에서 한 line은 [제목 '\t' 초록]의 형식을 가진다. 

본 함수는 제목과 초록 각각을 normalize() 한 뒤, 이를 단어 단위로 나누어 반환한다. 

예를 들어, " 'Hi, I'm Hyowon'  '\t' 'Hi, Hyowon!' "이라는 하나의 line은 두 개의 리스트에 담기게 된다. 

['hi', 'i', ''m', 'hyowon'] ['hi', 'hyowon']

src는 source의 준말로, 제목을 의미하고, trg은 target의 준말로, 초록을 의미한다.

- build_vocab()은 데이터에 있는 단어들을 입력으로 받아, 이를 빈도 기준으로 정렬한다. 

그 뒤, 위에서 선언한 unk, sos, eos와 합친다. 

결과적으로 0번: unk, 1번: sos, 2번: eos, 3번: 'this' ....... 를 얻을 수 있다. 

이는 컴퓨터가 인간 언어를 이해할 수 없으므로, 숫자로 바꾸어주는 과정이다.

- word2tensor() 함수는 3가지 기능을 수행한다. 

1. input데이터의 시작과 끝에 sos와 eos를 붙이고, output에는 eos를 붙인다. 
2. 입력받은 단어들을 매칭되는 숫자로 바꾸어준다. 
3. max_len을 기준으로 모든 초록과 제목의 크기를 통일한다. 

    이를 padding이라고 부른다. 0을 추가로 넣어 길이를 맞춘다. 

반환하는 것은: 정수화된 데이터와 그 데이터의 본래 길이이다.

## PairDataset

In [ ]:
class PairDataset(Dataset):
  def __init__(self, path, max_len=300):
    def filter_pair(p):
      return not (len(p[0]) > max_len or len(p[1]) > max_len)

    with open(path) as fp:
      pairs = map(parse_line, fp)
      pairs = filter(filter_pair, pairs)
      pairs = list(pairs)
    
    pairs = sorted(pairs, key=lambda p:len(p[0]),reverse=True)
    src = [p[0] for p in pairs]
    trg = [p[1] for p in pairs]

    self.src_word_list, self.src_word_dict = build_vocab(
        itertools.chain.from_iterable(src)
    )
    self.trg_word_list, self.trg_word_dict = build_vocab(
        itertools.chain.from_iterable(trg)
    )

    self.src_data = [word2tensor(
        words, self.src_word_dict, max_len) for words in src]
    self.trg_data = [word2tensor(
        words, self.trg_word_dict, max_len, sos=True) for words in trg]

  def __len__(self):
      return len(self.src_data)
      
  def __getitem__(self, idx):
      src, lsrc = self.src_data[idx]
      trg, ltrg = self.trg_data[idx]
      return src, lsrc, trg, ltrg

PairDataset class는 위에 설명한 함수들을 이용해 전처리를 하고, 원하는 값들을 반환해준다. 

- Data format에서 만든 파일을 불러와, max_len이 넘는 자료는 버린다. 해당 코드에서는 300으로 설정했다.
- 모든 데이터를 제목의 길이를 기준으로 정렬한다.

     이는 padding값(0)을 모델의 학습에서 고려하지 않도록 하는 메서드인 pack_padded_sequence()를 사용하기 위함이다. 

      만약, padding값이 학습에 들어간다면, 가장 많이 나오는 '0'이 다음에 나올 확률이 가장 높은 단어가 될 것이다. 이는 문장 생성의 정확성을 낮추는 결정적인 원인이 된다. 

- 제목과 초록 각각의 단어 사전을 만든다.
- word2tensor()를 이용해 모든 제목과 초록을 정수화한다.
- __len__()는 데이터 전체의 크기를 반환한다.
- __getitem__()는 제목, 초록의 정수화된 리스트와 각각의 길이를 반환한다.

    lsrc, ltrg은 length를 의미한다. 

self.src_data = 정수화된 문장 텐서 + 문장의 길이(int)를 담고있는 **list** self.trg_data = 정수화된 문장 텐서 + 문장의 길이(int)를 담고있는 **list**

** PyTorch에서는 tensor를 사용한다. tensorflow에서의 tensor의 개념과 동일하다. PyTorch에서 수를 다루기 위해서는 텐서화를 시켜야한다.  

앞으로는 좀 더 정확한 표현을 위해 정수화가 아닌 텐서화라고 명명한다.

In [ ]:
batch_size = 8
max_len = 300
path = "/content/abstract_train_10000.txt"
ds = PairDataset(path, max_len=max_len)
loader = DataLoader(ds, batch_size=batch_size, shuffle=False, num_workers=4)

위의 PairDataset을 선언한 이유는 DataLoader()를 사용하기 위함이다. 

Dataloader는 모델의 훈련이 이루어질 때, batch 단위로 나누어진 데이터를 순차적으로 제공한다. 

이때, batch란 간단하게 한 번에 학습하는 것은 비효율적이니 나누어 학습하기 위한 단위라고 생각하면 된다.

In [ ]:
for i in loader:
  print(i[0].shape, i[1].shape, i[2].shape, i[3].shape)
  break
# 잘 들어왔다!

torch.Size([8, 301]) torch.Size([8]) torch.Size([8, 301]) torch.Size([8])


# Seq2seq & Attetion

시간 정보가 중요한 데이터를 **시계열 데이터**라고 한다. 

seq2seq는 시퀀스 투 시퀀스의 준말로, 시계열 데이터를 다루기 위한 모델이다. 

하나의 시퀀스를 입력으로 받아, 다른 시퀀스를 출력한다. 

대표적인 예시로는 번역기가 있다. 

기본적인 seq2seq의 구조는 다음과 같다.

![대체 텍스트](https://i.imgur.com/GokIBvj.png)

보다시피, encoder-decoder의 구조로 이루어져있다는 것을 알 수 있다. 

encoder가 입력 문장에 대한 정보를 decoder에게 넘기면, decoder가 이를 기반으로 문장을 생성한다. 

이때, encoder가 decoder에게 넘기는 문장 정보를 **context vector**라고 한다. 

하지만, 단순한 encoder-decoder모델은 문장의 길이가 길어질수록 정확한 정보를 전달할 수 없다는 단점이 있다. 

오로지 한 번만 context vector를 전달하기 때문이다. 

이를 보완하기 위해 등장한 것이 **Attention****이다.

![대체 텍스트](https://i.imgur.com/kLPvBAr.png)

이처럼 Attention기법은 decoder의 모든 cell로 하여금 context vector를 받을 수 있게 한다. decoder는 context vector에서 자신이 어떠한 정보에 집중(*어텐션* ) 해야하는지 수식을 통해 판단한다. 

우리가 사용한 모델을 한 문장으로 정의하면 다음과 같다:

> **Seq2seq with Attention, bidirectional GRU and teacher forcing**

## Encoder

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim,dropout=0.2):
        super().__init__()

        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout

        self.embedding = nn.Embedding(input_dim, 
                                      emb_dim, 
                                      padding_idx=0)
        
        self.gru = nn.GRU(emb_dim,enc_hid_dim,batch_first=True,bidirectional=True)
        
        # forward gru + backward gru -> enc_hid_dim*2
        self.linear = nn.Linear(enc_hid_dim*2, dec_hid_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self,input_sen,input_length, padding_length=301):
        
        embedded = self.embedding(input_sen)
        
        packed = nn.utils.rnn.pack_padded_sequence(embedded,input_length, batch_first=True)

        outputs, h = self.gru(packed)
        
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs, batch_first = True, total_length = padding_length)

        hidden = torch.tanh(self.linear(torch.cat((h[-2,:,:], h[-1,:,:]), dim = 1)))

        return outputs, hidden

Encoder는 5가지를 인자로 받는다 : 

입력 문장의 차원, 임베딩 차원, 인코더의 hidden 차원, 디코더의 hidden 차원, dropout 지수

- '입력 문장의 차원'은 제목_단어사전의 길이와 동일하다.
- '임베딩 차원'은 입력 문장의 차원을 축소할 크기를 의미한다.

    (embedding의 개념은 궁금하면 검색해보세용)

- 'dropout'은 과적합을 막기 위해, 학습시 버리는 데이터의 비율을 의미한다.

- **bidirectional GRU**

    RNN의 일종인 GRU를 사용했다. bidirectional = True로 설정하여, 앞에서부터와 뒤에서부터의 정보 모두를 고려했다. 

Encoder의 실질적인 동작은 forward()에 기술되어있다. 

1. 입력 문장을 embedding layer에 통과시킨다. 

    차원변화: [batch size, src_sent_len] ——→ [batch size, src_sent_len, emb dim]

2. pack_padded_sequence()를 이용해 임베딩문장을 변형한다. 
3. bidirectional GRU layer에 이를 통과시킨다. 
4. pad_packed_sequence()를 이용해 pack한 문장을 다시 풀어준다. 
5. linear layer에 통과시킴으로 양방향 정보를 종합한다.



## Attention

In [ ]:
class Attention(nn.Module):
  def __init__(self, enc_hid_dim, dec_hid_dim):
      super().__init__()
      
      self.enc_hid_dim = enc_hid_dim
      self.dec_hid_dim = dec_hid_dim
      
      self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
      self.v = nn.Parameter(torch.rand(dec_hid_dim))
      
  def forward(self, hidden, context, mask):
      
      #hidden = [batch size, dec hid dim]
      #context = [batch size, sen len, enc hid dim * 2]

      batch_size = context.shape[0]
      src_len = context.shape[1]

      #repeat encoder hidden state src_len times
      hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
      
      context = context.permute(0, 1, 2)

      # hidden = [batch size, src sent len, dec hid dim]
      # context = [batch size, src sent len, enc hid dim * 2]
      
      sim = torch.tanh(self.attn(torch.cat((hidden, context), dim = 2))) 
      
      sim = sim.permute(0, 2, 1)
      
      #sim = [batch size, dec hid dim, sen len]
      
      v = self.v.repeat(batch_size, 1).unsqueeze(1)
      # v = [batch size, 1, dec hid dim]
              
      attention = torch.bmm(v, sim).squeeze(1)
      
      #attention= [batch size, src len]
      attention = attention.masked_fill(mask == 0, -1e10)
      
      return F.softmax(attention, dim=1)

## Decoder

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()
        
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.output_dim = output_dim
        self.dropout = dropout
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)   
        self.gru = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        self.out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        self.dropout = nn.Dropout(dropout)


    def forward(self, input_tok, hidden, context, mask):

        # input_tok = [batch size]
        # hidden = [batch size, dec hid dim]
        
        input_tok = input_tok.unsqueeze(0)
        embedded = self.dropout(self.embedding(input_tok))

        a = self.attention(hidden, context, mask)  
        a = a.unsqueeze(1)
        # a = [batch size, 1, src len]

        context = context.permute(0, 1, 2)
        # context = [batch size, src len, enc hid dim * 2]
        
        weighted = torch.bmm(a, context)
        weighted = weighted.permute(1, 0, 2)
        #weighted = [1, batch size, enc hid dim * 2]

        gru_input = torch.cat((embedded, weighted), dim = 2)
        # gru_input = [1, batch size, (enc hid dim * 2) + emb dim]

        output,h = self.gru(gru_input, hidden.unsqueeze(0))

        assert (output == h).all()

        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        output = self.out(torch.cat((output, weighted, embedded), dim = 1))
        
        return output, h.squeeze(0), a.squeeze(1)

decoder에서는 우선, encoder에서 받은 context vector를 attention layer에 통과시킨다. 

이를 통해서, 어느 시점에 어디에 집중해야하는지를 의미하는 가중행렬을 얻을 수 있다. 

이를 one-direction GRU layer에 통과시킨뒤, 결과값을 내보내는 linear layer에 통과시킨다.

## Seq2Seq

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, pad_idx, sos_idx, eos_idx, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.pad_idx = pad_idx
        self.sos_idx = sos_idx
        self.eos_idx = eos_idx
        self.device = device
    
    def create_mask(self, src):
        mask = (src != self.pad_idx)
        return mask

    def forward(self, src, lsrc,trg, teacher_forcing_ratio = 0.75):

        batch_size = trg.shape[0]
        max_len = trg.shape[1]
        trg_vocab_size = len(ds.trg_word_list)
        
        #tensor to store
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        attentions = torch.zeros(max_len, batch_size, src.shape[1]).to(self.device)

        context, hidden = self.encoder(src, lsrc)

        mask = self.create_mask(src)

        use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
        
        #first input = <sos>
        inputs = trg[:,0]

        result = []    
        for t in range(1, max_len):
          output, hidden, attention = self.decoder(inputs, hidden, context, mask)
          outputs[t] = output
          attentions[t] = attention

          if use_teacher_forcing:
            inputs = trg[:,t]
          else:
            top1 = output.argmax(1) 
            inputs = top1

          result.append(ds.trg_word_list[inputs[0].item()])

          if inputs[0].item() == self.eos_idx: break

        return outputs, attentions, result


encoder와 decoder, attention을 모두 포함하는 seq2seq class이다. 

먼저 attention을 위한 mask를 생성하는 함수가 선언되어있다. 

forwrad()를 분석하면 다음과 같다:

1. output문장과 attention값을 담을 빈 텐서를 선언한다. 

    값을 담을 공간을 마련해주는 것이다. 

2. encoder에 입력 문장을 넣어 context vector를 얻는다. 
3. attention을 위한 mask를 생성한다. 
4. sos 토큰을 decoder 입력의 첫번째 값으로 선언한다. 
5. 한 단어를 넣었을 때, 다음 단어를 예측한다. (이를 max_len만큼 반복한다)

    앞의 단어들과 attention value에 근거해, 다음에 나올 확률이 가장 높은 단어가 예측값이다.  

6. 잘못된 예측으로 인해 모든 뒷 단어들이 잘못되는 경우를 막기 위해, teacher forcing을 사용한다. 

- teacher forcing

다음 decoder cell의 입력으로 이전의 decoder cell에서의 예측값이 아닌,  실제 정답값을 입력으로 넣는 것을 의미한다.

## Process

In [ ]:
INPUT_DIM = len(ds.src_word_list)
OUTPUT_DIM = len(ds.trg_word_list)

ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.2
DEC_DROPOUT = 0.2

PAD_IDX = 0
SOS_IDX = 1
EOS_IDX = 2

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda:0" if USE_CUDA else "cpu")

model = Seq2Seq(enc, dec, PAD_IDX, SOS_IDX, EOS_IDX, device).to(device)
# model = nn.DataParallel(model)

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=0)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(10152, 256, padding_idx=0)
    (gru): GRU(256, 512, batch_first=True, bidirectional=True)
    (linear): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
    )
    (embedding): Embedding(20643, 256)
    (gru): GRU(1280, 512)
    (out): Linear(in_features=1792, out_features=20643, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
)

In [ ]:
epoch_iter = 8

for epoch in range(epoch_iter):
  model.train()
  epoch_loss = 0
  for pairs in loader:
        
        src = pairs[0].to(device)
        lsrc = pairs[1].to(device)
        trg = pairs[2].to(device)
        ltrg = pairs[3]
        

        optimizer.zero_grad()
        
        output,attention, result = model(src, lsrc, trg)

        #output = [batch size,trg sen len, output dim]
        #trg = [batch size,trg sen len]

        output = output[1:].view(-1, output.shape[-1])
        trg = trg[:, 1:, ]
        trg = trg.reshape(-1)

        # output = [(trg sent len - 1) * batch size, output dim]
        # trg = [(trg sent len - 1) * batch size]

        loss = criterion(output, trg)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(),1)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        print("epoch: ", epoch, "\nloss: ", loss.item(),"\nresult: \n", result)       


epoch:  0 
loss:  9.935256958007812 
result: 
 ['we', 'propose', 'a', 'domain', 'specific', 'model', 'for', 'statistical', 'machine', 'translation', '.', 'it', 'is', 'wellknown', 'that', 'domain', 'specific', 'language', 'models', 'perform', 'well', 'in', 'automatic', 'speech', 'recognition', '.', 'we', 'show', 'that', 'domain', 'specific', 'language', 'and', 'translation', 'models', 'also', 'benefit', 'statistical', 'machine', 'translation', '.', 'however', 'there', 'are', 'two', 'problems', 'with', 'using', 'domain', 'specific', 'models', '.', 'the', 'first', 'is', 'the', 'data', 'sparseness', 'problem', '.', 'we', 'employ', 'an', 'adaptation', 'technique', 'to', 'overcome', 'this', 'problem', '.', 'the', 'second', 'issue', 'is', 'domain', 'prediction', '.', 'in', 'order', 'to', 'perform', 'adaptation', 'the', 'domain', 'must', 'be', 'provided', 'however', 'in', 'many', 'cases', 'the', 'domain', 'is', 'not', 'known', 'or', 'changes', 'dynamically', '.', 'for', 'these', 'cases', 'not'

In [ ]:
# 용량 문제로 아직 여기까지x

model.eval()
epoch_loss = 0

with torch.no_grad():
  for pairs in loader:
        
        src = pairs[0].to(device)
        lsrc = pairs[1].to(device)
        trg = pairs[2].to(device)
        output,attention, result = model(src, lsrc, trg,0)

        output = output[1:].view(-1, output.shape[-1])
        trg = trg[:, 1:, ]
        trg = trg.reshape(-1)

        loss = criterion(output, trg)
        epoch_loss += loss.item()
